In [16]:
###########################################################################################################################
#
# Antonio Vieira da Silva Neto - NUSP 5690149
#
# Doutorado em Ciências
# Escola Politécnica da Universidade de São Paulo
# Programa de Pós-Graduação em Engenharia Elétrica
# Departamento de Engenharia de Computação e Sistemas Digitais
# Área de Concentração: Confiabilidade e Segurança
#
# Tese de Doutorado: Safety ArtISt: Um Método para a Garantia de Segurança Crítica de Sistemas com Inteligência Artificial
#
# Estudo de Caso 1 - Sistema de Detecção de Arritmias Cardíacas
#
# Arquivo para Pré-Processamento Exploratório das Bases de Dados "mitbit_train.csv" e "mitbih_test.csv".
#
###########################################################################################################################

In [17]:
###########################################################################################################################
#
# Tabela de Controle de Versões do Código
#
#--------|-------------------|------------------------------|--------------------------------------------------------------
# Versão | Data (dd/mm/aaaa) | Autor                        | Descrição da Versão
#--------|-------------------|------------------------------|--------------------------------------------------------------
#   01   |    30/05/2022     | Antonio Vieira da Silva Neto | Versão inicial.
#------- |-------------------|------------------------------|--------------------------------------------------------------
#   02   |    01/06/2022     | Antonio Vieira da Silva Neto | Versão atualizada incluindo cálculos, globais e por categori- 
#        |                   |                              | a, sobre tamanho médio dos batimentos e distribuição do tama-
#        |                   |                              | nho dos batimentos e perfil médio dos batimentos.
#------- |-------------------|------------------------------|--------------------------------------------------------------
#
###########################################################################################################################

In [18]:
###########################################################################################################################
# Descrição Geral: O processamento exploratório das bases de dados "mitbih_train.csv" e "mitbih_test.csv" contempla as se-
# guintes etapas:
#
# 1) Avalia-se a distribuição de zeros em cada uma das 187 amostras que compõem cada registro de dados;
# 2) Avalia-se a distribuição de uns em cada uma das 187 amostras que compõem cada registro de dados;
# 3) Avalia-se se todos os registros foram categorizados com números inteiros válidos (i.e., entre 0 e 4) na coluna 188;
# 4) Avalia-se se há dados diferentes de quaisquer números no intervalo [0; 1] em cada uma das 187 amostras que compõem cada
#    registro de dados;
# 5) Avalia-se a distribuição do tamanho real de cada batimento cardíaco. Batimentos mais curtos do que 187 amostras são
#    contabilizados pela frequência de zeros consecutivos a partir de dado ponto do registro;
# 6) Avalia-se o tamanho médio dos batimentos cardíacos na base toda e em recortes por categoria;
# 7) Avalia-se a distribuição do tamanho dos batimentos cardíacos na base toda e em recortes por categoria;
# 8) Avaliam-se os perfis médios dos batimentos cardíacos na base toda e em recortes por categoria em duas situações: (i.)
#    considerando todas as amostras dos batimentos e (ii.) ponderando apenas as amostras válidas (i.e., ignorando as preen-
#    chidas artificialmente com valor 0, conforme detectado pela regra do item "5)").
#
# Cada um dos passos é executado com cada um dos arquivos mediante acionamento do procedimento "exploreDataEC1".
###########################################################################################################################

In [19]:
###########################################################################################################################
# Importação de Bibliotecas
###########################################################################################################################

import pandas as pd
import numpy as np

In [20]:
###########################################################################################################################
# Procedimento "exploreDataEC1"
#
# Objetivo: Realizar o processamento exploratório de um arquivo "*.csv" identificado na entrada "filepath" conforme a descrição
#           geral do módulo.
#
# Parâmetros de Entrada:
# - filepath: Endereço completo de um arquivo ".csv" a ser explorado.
#
# Parâmetros de Saída:
# Não há. Todas as saídas necessárias são impressas na tela do Jupyter Notebook durante a execução deste software.
###########################################################################################################################

def exploreDataEC1 (filepath):

    # Criação dos rótulos das colunas dos dados. São, ao todo, 187 amostras de ECGs e uma classificação final sobre o
    # batimento cardíaco.
    
    # Iniciação da Variável "dataCategories" como uma lista vazia
    dataCategories = []
    
    # Inclui em "dataCategories" as Strings "Sample1" até "Sample187"
    for i in range (1, 188, 1):
        
        dataCategories.append("Sample"+('{0:03}'.format(i)))
    
    # Inclui em "dataCategories" a String referente à categoria do batimento cardíaco ("BeatCategory")
    dataCategories.append("BeatCategory")
    
    # Lê arquivo CSV a partir do arquivo de origem. Como o primeiro argumento (obrigatório) da função "read_csv" é o endereço,
    # o parâmetro correspondente de read_csv (filepath_or_buffer) é apontado como sendo a entrada "filepath". "header" é atri-
    # buído como zero porque não há informações de cabeçalho no arquivo de origem.
    dataset = pd.read_csv(filepath, names = dataCategories, header = None)
    
    # Lê quantidades de linhas e colunas do arquivo
    nRows, nColumns = dataset.shape    
    print(f'There are {nRows} rows and {nColumns} columns')
    
    # 1) Contabiliza quantidade de zeros por amostra do batimento (coluna)
    print('Number of Zeros per Column / Heartbeat:')
        
    for col in range (0, 187, 1):
    
        numZeros = (dataset[dataCategories[col]] == 0).sum()
        print(f'{dataCategories[col]}: {numZeros}')
    
    # 2) Contabiliza quantidade de uns por amostra do batimento (coluna)
    print('Number of Ones per Column / Heartbeat:')
    
    for col in range (0, 187, 1):
    
        numOnes = (dataset[dataCategories[col]] == 1).sum()
        print(f'{dataCategories[col]}: {numOnes}')
        
    # 3) Contabiliza quantidade de classificações de batimentos por amostra do batimento (coluna)
    print('Number of Beat Categories per Line:')
    
    numZeros  = (dataset[dataCategories[187]] == 0).sum()
    print(f'Beat Category 0: {numZeros}')
    
    numOnes   = (dataset[dataCategories[187]] == 1).sum()
    print(f'Beat Category 1: {numOnes}')
    
    numTwos   = (dataset[dataCategories[187]] == 2).sum()
    print(f'Beat Category 2: {numTwos}')
    
    numThrees = (dataset[dataCategories[187]] == 3).sum()
    print(f'Beat Category 3: {numThrees}')
    
    numFours  = (dataset[dataCategories[187]] == 4).sum()
    print(f'Beat Category 4: {numFours}')
    
    numOthers = nRows - numZeros - numOnes - numTwos - numThrees - numFours
    print(f'Other Beat Categories: {numOthers}')
    
    # 4) Contabiliza quantidade de amostras fora do intervalo [0; 1] em cada uma das 187 amostras dos dados
    print('Number of Samples Outside [0; 1] per Column / Heartbeat:')
        
    for col in range (0, 187, 1):
        
        # Inicia contagem de números fora de itervalo por coluna como zero
        numOutsideRange = 0
        
        # Percorre cada linha da base de dados
        for row in range (0, nRows, 1):
            
            # Se o elemento da tabela estiver fora do intervalo [0 ;1], incrementa numOutsideRange em uma unidade
            if (dataset.iat[row, col] < 0 or dataset.iat[row, col] > 1):
            
                numOutsideRange = numOutsideRange + 1
            
        print(f'{dataCategories[col]}: {numOutsideRange}')
        
    # 5) Detecta Número de Amostras de Cada Batimento
    # Estratégia:
    # a) Se a última amostra for diferente de zero, a quantidade de amostras é 187;
    # b) Se a última amostra for igual a zero, as amostras são percorridas em ordem inversa até que uma amostra diferente de
    #    zero seja detectada. Quando isso ocorrer, a quantidade de amostras passa a ser a do último zero.
    #
    # Os tamanhos calculados nesta etapa são armazenados na lista 'numSamples' para serem usados nos passos "6)" a "8)"
    
    # Contabiliza quantidade de amostras fora do intervalo [0; 1] em cada uma das 187 amostras dos dados
    print('Number of Valid Samples per Heartbeat:')
    
    # Inicia lista "numSamplesRow" como lista vazia
    numSamples = []
    
    # Percorre cada linha da tabela de dados para obter os tamanhos dos batimentos cardíacos em número de amostras.
    for row in range (0, nRows, 1):
        
        # Cenário "a)"
        if (dataset.iat[row, 186] != 0):
            numSamplesRow = 187
        
        # Cenário "b)"
        else:
        
            for col in range (186, -1, -1):
            
                if (dataset.iat[row, col] != 0):
                    # Quantidade de amostras válidas correspondem ao valor de "col" acrescido de uma unidade
                    numSamplesRow = col + 1
                    break
        
        # Imprime tamanho de cada linha e adiciona à lista "numSamplesRow"
        print(f'Row {row}: {numSamplesRow}')
        numSamples.append(numSamplesRow)
        
    # 6) e 7) Calcula tamanho médio e distribuição de tamanho dos batimentos cardíacos da base de dados.
    # Os resultados gerados servem para a base de dados inteira e por recorte de categoria.    
    
    # Iniciação de contadores de tamanho por categoria
    contCat0   = np.zeros(188, int)
    contCat1   = np.zeros(188, int)
    contCat2   = np.zeros(188, int)
    contCat3   = np.zeros(188, int)
    contCat4   = np.zeros(188, int)
    contGlobal = np.zeros(188, int)
    
    # Iniciação de acumuladores de tamanho por categoria
    accumSizeCat0   = 0
    accumSizeCat1   = 0
    accumSizeCat2   = 0
    accumSizeCat3   = 0
    accumSizeCat4   = 0
    accumSizeGlobal = 0
    
    # Percorre-se cada linha da tabela e soma-se uma unidade a cada posição de "contCatX" e "contGlobal" o tamanho do
    # batimento cardíaco. Os acumuladores são incrementados com o tamanho de amostras da linha
    for row in range (0, nRows, 1):
        
        contGlobal[numSamples[row]] = contGlobal[numSamples[row]] + 1
        accumSizeGlobal = accumSizeGlobal + numSamples[row]
        
        if dataset.iat[row, 187] == 0:
            contCat0[numSamples[row]] = contCat0[numSamples[row]] + 1
            accumSizeCat0 = accumSizeCat0 + numSamples[row]
        
        elif dataset.iat[row, 187] == 1:
            contCat1[numSamples[row]] = contCat1[numSamples[row]] + 1
            accumSizeCat1 = accumSizeCat1 + numSamples[row]
        
        elif dataset.iat[row, 187] == 2:
            contCat2[numSamples[row]] = contCat2[numSamples[row]] + 1
            accumSizeCat2 = accumSizeCat2 + numSamples[row]

        elif dataset.iat[row, 187] == 3:
            contCat3[numSamples[row]] = contCat3[numSamples[row]] + 1
            accumSizeCat3 = accumSizeCat3 + numSamples[row]
        
        elif dataset.iat[row, 187] == 4:
            contCat4[numSamples[row]] = contCat4[numSamples[row]] + 1
            accumSizeCat4 = accumSizeCat4 + numSamples[row]
        
        else:
            print("Error! Invalid data category was found!")
            
    # 6) Imprime médias de número de amostras por categoria de batimento
    print('Average number of samples per heartbeat category:')
    print(f'Global database:{accumSizeGlobal / nRows}')
    print(f'Category 0:{accumSizeCat0 / numZeros}')
    print(f'Category 1:{accumSizeCat1 / numOnes}')
    print(f'Category 2:{accumSizeCat2 / numTwos}')
    print(f'Category 3:{accumSizeCat3 / numThrees}')
    print(f'Category 4:{accumSizeCat4 / numFours}')
    
    # 7) Imprime distribuições de número de amostras por categoria de batimento.
    print('Distribution of the number of heartbeat samples per heartbeat category (data ordered within range 0-187):')
    print(f'Global database:{contGlobal}')
    print(f'Category 0:{contCat0}')
    print(f'Category 1:{contCat1}')
    print(f'Category 2:{contCat2}')
    print(f'Category 3:{contCat3}')
    print(f'Category 4:{contCat4}')
    
    # 8) Determina perfil médio dos batimentos cardíacos globais e de cada categoria empregando duas abordagens:
    # (i.) todas as amostras dos batimentos e (ii.) ponderando apenas as amostras válidas (i.e., ignorando as preen-
    # chidas artificialmente com valor 0, conforme detectado pela regra do item "5)").
    # Como as amostras preenchidas artificialmente com valor "0" não alteram o valor acumulado da soma das amostras,
    # basta um único acumulador de amostras para cada categoria.
    
    # Iniciação de acumuladores com os valores de cada amostra para cada categoria
    accumBeatGeneral = np.zeros(187)
    accumBeatCat0    = np.zeros(187)
    accumBeatCat1    = np.zeros(187)
    accumBeatCat2    = np.zeros(187)
    accumBeatCat3    = np.zeros(187)
    accumBeatCat4    = np.zeros(187)
    
    # Variáveis auxiliares para contabilizar quantidade de linhas com amostras válidas por amostra
    contValidSamplesGeneral = np.zeros(187, int)
    contValidSamplesCat0    = np.zeros(187, int)
    contValidSamplesCat1    = np.zeros(187, int)
    contValidSamplesCat2    = np.zeros(187, int)
    contValidSamplesCat3    = np.zeros(187, int)
    contValidSamplesCat4    = np.zeros(187, int)
    
    # Percorre-se cada linha da tabela e, na sequência, cada coluna da linha. A cada valor visitado, soma-se o valor da
    # amostra da coluna nos acumuladores correspondentes: accumBeatGeneral sempre, e accumBeatCatX para uma linha pertencente
    # à categoria X. As linhas são percorridas até o final dos seus conteúdos válidos para otimizar desempenho.
    for row in range (0, nRows, 1):
        
        # A última coluna, correspondente ao valor da classificação, não é visitada.
        for column in range (0, numSamples[row], 1):
            
            # Incrementos de "accumBeatGeneral" e "contValidSamplesGeneral"
            accumBeatGeneral[column] = accumBeatGeneral[column] + dataset.iat[row, column]
            contValidSamplesGeneral[column] = contValidSamplesGeneral[column] + 1
            
            # Incrementos de "accumBeatCat0" e "contValidSamplesCat0" quando a linha é da categoria 0
            if dataset.iat[row, 187] == 0:                
                accumBeatCat0[column] = accumBeatCat0[column] + dataset.iat[row, column]
                contValidSamplesCat0[column] = contValidSamplesCat0[column] + 1
        
            # Incrementos de "accumBeatCat1" e "contValidSamplesCat1" quando a linha é da categoria 1
            elif dataset.iat[row, 187] == 1:                
                accumBeatCat1[column] = accumBeatCat1[column] + dataset.iat[row, column]
                contValidSamplesCat1[column] = contValidSamplesCat1[column] + 1
            
            # Incrementos de "accumBeatCat2" e "contValidSamplesCat2" quando a linha é da categoria 2
            elif dataset.iat[row, 187] == 2:                
                accumBeatCat2[column] = accumBeatCat2[column] + dataset.iat[row, column]
                contValidSamplesCat2[column] = contValidSamplesCat2[column] + 1
            
            # Incrementos de "accumBeatCat3" e "contValidSamplesCat3" quando a linha é da categoria 3
            elif dataset.iat[row, 187] == 3:                
                accumBeatCat3[column] = accumBeatCat3[column] + dataset.iat[row, column]
                contValidSamplesCat3[column] = contValidSamplesCat3[column] + 1
            
            # Incrementos de "accumBeatCat4" e "contValidSamplesCat4" quando a linha é da categoria 4
            elif dataset.iat[row, 187] == 4:                
                accumBeatCat4[column] = accumBeatCat4[column] + dataset.iat[row, column]
                contValidSamplesCat4[column] = contValidSamplesCat4[column] + 1
            
            # Cenário inesperado de erro.
            else:
                print("Error! Invalid data category was found!")
                
    
    # 8-(i.) Imprime perfil médio dos batimentos cardíacos da base de dados considerando todas as amostras
    print('Average heartbeat profile per heartbeat category for full 187-sample set:')
    print(f'Global database:{accumBeatGeneral / nRows}')
    print(f'Category 0:{accumBeatCat0 / numZeros}')
    print(f'Category 1:{accumBeatCat1 / numOnes}')
    print(f'Category 2:{accumBeatCat2 / numTwos}')
    print(f'Category 3:{accumBeatCat3 / numThrees}')
    print(f'Category 4:{accumBeatCat4 / numFours}')
    
    # 8-(ii.) Imprime perfil médio dos batimentos cardíacos da base de dados considerando apenas amostras válidas
    print('Average heartbeat profile per heartbeat category for valid samples set:')
    print(f'Global database:{accumBeatGeneral / contValidSamplesGeneral}')
    print(f'Category 0:{accumBeatCat0 / contValidSamplesCat0}')
    print(f'Category 1:{accumBeatCat1 / contValidSamplesCat1}')
    print(f'Category 2:{accumBeatCat2 / contValidSamplesCat2}')
    print(f'Category 3:{accumBeatCat3 / contValidSamplesCat3}')
    print(f'Category 4:{accumBeatCat4 / contValidSamplesCat4}')                

In [21]:
# Executa Análise Exploratória para o Arquivo "mitbih_test.csv"
# Localização do arquivo CSV: Mesmo diretório deste arquivo.
exploreDataEC1("./mitbih_test.csv")

There are 21892 rows and 188 columns
Number of Zeros per Column / Heartbeat:
Sample001: 653
Sample002: 162
Sample003: 407
Sample004: 2668
Sample005: 988
Sample006: 568
Sample007: 439
Sample008: 449
Sample009: 405
Sample010: 277
Sample011: 296
Sample012: 372
Sample013: 333
Sample014: 339
Sample015: 172
Sample016: 149
Sample017: 111
Sample018: 96
Sample019: 122
Sample020: 110
Sample021: 97
Sample022: 109
Sample023: 119
Sample024: 137
Sample025: 225
Sample026: 271
Sample027: 301
Sample028: 273
Sample029: 234
Sample030: 212
Sample031: 197
Sample032: 216
Sample033: 260
Sample034: 242
Sample035: 194
Sample036: 136
Sample037: 122
Sample038: 134
Sample039: 130
Sample040: 134
Sample041: 143
Sample042: 161
Sample043: 157
Sample044: 168
Sample045: 167
Sample046: 188
Sample047: 193
Sample048: 221
Sample049: 234
Sample050: 276
Sample051: 271
Sample052: 265
Sample053: 284
Sample054: 308
Sample055: 329
Sample056: 366
Sample057: 395
Sample058: 435
Sample059: 473
Sample060: 504
Sample061: 549
Sample062

Sample173: 0
Sample174: 0
Sample175: 0
Sample176: 0
Sample177: 0
Sample178: 0
Sample179: 0
Sample180: 0
Sample181: 0
Sample182: 0
Sample183: 0
Sample184: 0
Sample185: 0
Sample186: 0
Sample187: 0
Number of Valid Samples per Heartbeat:
Row 0: 124
Row 1: 100
Row 2: 109
Row 3: 105
Row 4: 115
Row 5: 106
Row 6: 91
Row 7: 116
Row 8: 129
Row 9: 100
Row 10: 97
Row 11: 103
Row 12: 82
Row 13: 118
Row 14: 151
Row 15: 135
Row 16: 99
Row 17: 80
Row 18: 88
Row 19: 66
Row 20: 151
Row 21: 165
Row 22: 148
Row 23: 80
Row 24: 170
Row 25: 144
Row 26: 100
Row 27: 78
Row 28: 107
Row 29: 133
Row 30: 68
Row 31: 77
Row 32: 106
Row 33: 87
Row 34: 69
Row 35: 86
Row 36: 120
Row 37: 101
Row 38: 181
Row 39: 79
Row 40: 148
Row 41: 108
Row 42: 152
Row 43: 146
Row 44: 121
Row 45: 164
Row 46: 96
Row 47: 115
Row 48: 106
Row 49: 110
Row 50: 108
Row 51: 133
Row 52: 100
Row 53: 162
Row 54: 89
Row 55: 79
Row 56: 91
Row 57: 107
Row 58: 37
Row 59: 152
Row 60: 143
Row 61: 66
Row 62: 97
Row 63: 183
Row 64: 133
Row 65: 76
Row 66:

Row 663: 88
Row 664: 80
Row 665: 93
Row 666: 157
Row 667: 133
Row 668: 148
Row 669: 97
Row 670: 103
Row 671: 100
Row 672: 136
Row 673: 131
Row 674: 122
Row 675: 107
Row 676: 112
Row 677: 100
Row 678: 112
Row 679: 119
Row 680: 97
Row 681: 139
Row 682: 98
Row 683: 79
Row 684: 120
Row 685: 91
Row 686: 143
Row 687: 113
Row 688: 146
Row 689: 113
Row 690: 131
Row 691: 65
Row 692: 126
Row 693: 83
Row 694: 83
Row 695: 130
Row 696: 96
Row 697: 132
Row 698: 81
Row 699: 109
Row 700: 111
Row 701: 102
Row 702: 140
Row 703: 132
Row 704: 116
Row 705: 85
Row 706: 150
Row 707: 85
Row 708: 77
Row 709: 116
Row 710: 118
Row 711: 127
Row 712: 170
Row 713: 91
Row 714: 92
Row 715: 104
Row 716: 107
Row 717: 126
Row 718: 138
Row 719: 79
Row 720: 136
Row 721: 118
Row 722: 106
Row 723: 99
Row 724: 121
Row 725: 102
Row 726: 129
Row 727: 120
Row 728: 105
Row 729: 156
Row 730: 160
Row 731: 93
Row 732: 93
Row 733: 94
Row 734: 114
Row 735: 108
Row 736: 79
Row 737: 74
Row 738: 108
Row 739: 69
Row 740: 106
Row 741: 137

Row 1301: 92
Row 1302: 99
Row 1303: 93
Row 1304: 98
Row 1305: 150
Row 1306: 129
Row 1307: 133
Row 1308: 95
Row 1309: 91
Row 1310: 90
Row 1311: 143
Row 1312: 83
Row 1313: 81
Row 1314: 87
Row 1315: 127
Row 1316: 116
Row 1317: 166
Row 1318: 102
Row 1319: 119
Row 1320: 102
Row 1321: 105
Row 1322: 105
Row 1323: 145
Row 1324: 134
Row 1325: 101
Row 1326: 80
Row 1327: 91
Row 1328: 88
Row 1329: 137
Row 1330: 142
Row 1331: 126
Row 1332: 109
Row 1333: 120
Row 1334: 89
Row 1335: 101
Row 1336: 125
Row 1337: 137
Row 1338: 98
Row 1339: 82
Row 1340: 150
Row 1341: 83
Row 1342: 138
Row 1343: 99
Row 1344: 110
Row 1345: 138
Row 1346: 168
Row 1347: 91
Row 1348: 100
Row 1349: 107
Row 1350: 88
Row 1351: 139
Row 1352: 100
Row 1353: 105
Row 1354: 109
Row 1355: 108
Row 1356: 126
Row 1357: 108
Row 1358: 160
Row 1359: 105
Row 1360: 98
Row 1361: 155
Row 1362: 89
Row 1363: 118
Row 1364: 100
Row 1365: 128
Row 1366: 75
Row 1367: 108
Row 1368: 91
Row 1369: 99
Row 1370: 108
Row 1371: 94
Row 1372: 175
Row 1373: 125
Row 

Row 1964: 107
Row 1965: 95
Row 1966: 107
Row 1967: 134
Row 1968: 81
Row 1969: 166
Row 1970: 99
Row 1971: 102
Row 1972: 70
Row 1973: 114
Row 1974: 77
Row 1975: 79
Row 1976: 55
Row 1977: 79
Row 1978: 132
Row 1979: 161
Row 1980: 117
Row 1981: 90
Row 1982: 109
Row 1983: 135
Row 1984: 141
Row 1985: 169
Row 1986: 94
Row 1987: 80
Row 1988: 99
Row 1989: 124
Row 1990: 99
Row 1991: 81
Row 1992: 100
Row 1993: 107
Row 1994: 117
Row 1995: 98
Row 1996: 102
Row 1997: 85
Row 1998: 104
Row 1999: 103
Row 2000: 104
Row 2001: 119
Row 2002: 143
Row 2003: 99
Row 2004: 25
Row 2005: 113
Row 2006: 131
Row 2007: 127
Row 2008: 152
Row 2009: 131
Row 2010: 110
Row 2011: 108
Row 2012: 127
Row 2013: 89
Row 2014: 170
Row 2015: 122
Row 2016: 157
Row 2017: 73
Row 2018: 98
Row 2019: 79
Row 2020: 102
Row 2021: 95
Row 2022: 130
Row 2023: 128
Row 2024: 136
Row 2025: 90
Row 2026: 98
Row 2027: 71
Row 2028: 93
Row 2029: 75
Row 2030: 131
Row 2031: 100
Row 2032: 80
Row 2033: 80
Row 2034: 92
Row 2035: 105
Row 2036: 118
Row 2037:

...

Row 20334: 71
Row 20335: 117
Row 20336: 115
Row 20337: 120
Row 20338: 115
Row 20339: 120
Row 20340: 121
Row 20341: 113
Row 20342: 118
Row 20343: 125
Row 20344: 116
Row 20345: 116
Row 20346: 129
Row 20347: 121
Row 20348: 104
Row 20349: 119
Row 20350: 118
Row 20351: 121
Row 20352: 118
Row 20353: 119
Row 20354: 108
Row 20355: 122
Row 20356: 120
Row 20357: 123
Row 20358: 123
Row 20359: 91
Row 20360: 122
Row 20361: 119
Row 20362: 80
Row 20363: 123
Row 20364: 114
Row 20365: 127
Row 20366: 117
Row 20367: 117
Row 20368: 113
Row 20369: 124
Row 20370: 118
Row 20371: 117
Row 20372: 119
Row 20373: 60
Row 20374: 119
Row 20375: 119
Row 20376: 116
Row 20377: 120
Row 20378: 114
Row 20379: 103
Row 20380: 116
Row 20381: 122
Row 20382: 116
Row 20383: 119
Row 20384: 121
Row 20385: 117
Row 20386: 119
Row 20387: 112
Row 20388: 116
Row 20389: 115
Row 20390: 128
Row 20391: 72
Row 20392: 117
Row 20393: 115
Row 20394: 128
Row 20395: 96
Row 20396: 118
Row 20397: 106
Row 20398: 120
Row 20399: 130
Row 20400: 127
R

Row 20894: 119
Row 20895: 120
Row 20896: 116
Row 20897: 124
Row 20898: 111
Row 20899: 107
Row 20900: 117
Row 20901: 120
Row 20902: 114
Row 20903: 117
Row 20904: 129
Row 20905: 116
Row 20906: 69
Row 20907: 126
Row 20908: 117
Row 20909: 124
Row 20910: 123
Row 20911: 123
Row 20912: 117
Row 20913: 116
Row 20914: 129
Row 20915: 120
Row 20916: 114
Row 20917: 119
Row 20918: 114
Row 20919: 124
Row 20920: 115
Row 20921: 125
Row 20922: 128
Row 20923: 121
Row 20924: 122
Row 20925: 123
Row 20926: 120
Row 20927: 126
Row 20928: 118
Row 20929: 122
Row 20930: 122
Row 20931: 122
Row 20932: 125
Row 20933: 123
Row 20934: 122
Row 20935: 105
Row 20936: 114
Row 20937: 106
Row 20938: 121
Row 20939: 125
Row 20940: 128
Row 20941: 79
Row 20942: 121
Row 20943: 121
Row 20944: 116
Row 20945: 118
Row 20946: 116
Row 20947: 119
Row 20948: 120
Row 20949: 121
Row 20950: 120
Row 20951: 104
Row 20952: 123
Row 20953: 104
Row 20954: 121
Row 20955: 118
Row 20956: 118
Row 20957: 123
Row 20958: 124
Row 20959: 117
Row 20960: 1

Row 21500: 117
Row 21501: 121
Row 21502: 122
Row 21503: 121
Row 21504: 123
Row 21505: 124
Row 21506: 119
Row 21507: 125
Row 21508: 124
Row 21509: 125
Row 21510: 117
Row 21511: 120
Row 21512: 119
Row 21513: 89
Row 21514: 127
Row 21515: 120
Row 21516: 115
Row 21517: 119
Row 21518: 118
Row 21519: 124
Row 21520: 117
Row 21521: 125
Row 21522: 103
Row 21523: 113
Row 21524: 121
Row 21525: 124
Row 21526: 119
Row 21527: 119
Row 21528: 121
Row 21529: 118
Row 21530: 115
Row 21531: 123
Row 21532: 118
Row 21533: 111
Row 21534: 119
Row 21535: 124
Row 21536: 115
Row 21537: 121
Row 21538: 116
Row 21539: 118
Row 21540: 123
Row 21541: 120
Row 21542: 121
Row 21543: 114
Row 21544: 123
Row 21545: 122
Row 21546: 119
Row 21547: 101
Row 21548: 125
Row 21549: 126
Row 21550: 118
Row 21551: 119
Row 21552: 121
Row 21553: 126
Row 21554: 120
Row 21555: 122
Row 21556: 119
Row 21557: 114
Row 21558: 119
Row 21559: 127
Row 21560: 119
Row 21561: 118
Row 21562: 124
Row 21563: 118
Row 21564: 123
Row 21565: 118
Row 21566: 

Average heartbeat profile per heartbeat category for full 187-sample set:
Global database:[0.89440994 0.7619022  0.42662662 0.22159611 0.20167607 0.20989054
 0.20480477 0.2009923  0.19763355 0.19602235 0.1979051  0.20243617
 0.20730265 0.21152391 0.21684016 0.22345477 0.22984513 0.23546266
 0.24070144 0.24597551 0.25175142 0.25766389 0.26378799 0.27054099
 0.27692127 0.2831786  0.28934201 0.29534597 0.30118001 0.30647774
 0.31170844 0.3162345  0.31986801 0.32281406 0.32471348 0.32581241
 0.32585629 0.32472877 0.3225182  0.31914743 0.31485499 0.3102251
 0.30554151 0.30098183 0.29634984 0.29153634 0.28678625 0.2827814
 0.27958411 0.27702675 0.27555911 0.27473737 0.27377783 0.27301665
 0.27263062 0.27230388 0.27187208 0.27192422 0.27250329 0.27299592
 0.27377041 0.27500059 0.27559133 0.27627313 0.27802068 0.28060829
 0.28134284 0.27901202 0.27528569 0.27061518 0.26632807 0.26295433
 0.25919989 0.25599609 0.25378663 0.25285458 0.25464312 0.25756804
 0.25957444 0.25880965 0.25719097 0.25610

C:\Users\Arnaldo\AppData\Local\Temp\ipykernel_2472\1435497785.py:275: RuntimeWarning: invalid value encountered in true_divide
  print(f'Category 3:{accumBeatCat3 / contValidSamplesCat3}')
C:\Users\Arnaldo\AppData\Local\Temp\ipykernel_2472\1435497785.py:276: RuntimeWarning: invalid value encountered in true_divide
  print(f'Category 4:{accumBeatCat4 / contValidSamplesCat4}')


In [22]:
# Executa Análise Exploratória para o Arquivo "mitbih_train.csv"
# Localização do arquivo CSV: Mesmo diretório deste arquivo.
exploreDataEC1("./mitbih_train.csv")

There are 87554 rows and 188 columns
Number of Zeros per Column / Heartbeat:
Sample001: 2778
Sample002: 689
Sample003: 1589
Sample004: 10871
Sample005: 4062
Sample006: 2230
Sample007: 1628
Sample008: 1818
Sample009: 1611
Sample010: 1178
Sample011: 1323
Sample012: 1368
Sample013: 1386
Sample014: 1227
Sample015: 734
Sample016: 524
Sample017: 474
Sample018: 469
Sample019: 425
Sample020: 400
Sample021: 366
Sample022: 389
Sample023: 390
Sample024: 556
Sample025: 792
Sample026: 1101
Sample027: 1162
Sample028: 951
Sample029: 871
Sample030: 777
Sample031: 742
Sample032: 793
Sample033: 948
Sample034: 965
Sample035: 679
Sample036: 497
Sample037: 418
Sample038: 414
Sample039: 414
Sample040: 453
Sample041: 473
Sample042: 514
Sample043: 523
Sample044: 562
Sample045: 615
Sample046: 644
Sample047: 680
Sample048: 755
Sample049: 843
Sample050: 953
Sample051: 955
Sample052: 972
Sample053: 1028
Sample054: 1144
Sample055: 1253
Sample056: 1401
Sample057: 1558
Sample058: 1712
Sample059: 1819
Sample060: 1950

Sample161: 0
Sample162: 0
Sample163: 0
Sample164: 0
Sample165: 0
Sample166: 0
Sample167: 0
Sample168: 0
Sample169: 0
Sample170: 0
Sample171: 0
Sample172: 0
Sample173: 0
Sample174: 0
Sample175: 0
Sample176: 0
Sample177: 0
Sample178: 0
Sample179: 0
Sample180: 0
Sample181: 0
Sample182: 0
Sample183: 0
Sample184: 0
Sample185: 0
Sample186: 0
Sample187: 0
Number of Valid Samples per Heartbeat:
Row 0: 100
Row 1: 137
Row 2: 95
Row 3: 96
Row 4: 140
Row 5: 120
Row 6: 131
Row 7: 87
Row 8: 126
Row 9: 80
Row 10: 95
Row 11: 140
Row 12: 98
Row 13: 97
Row 14: 109
Row 15: 83
Row 16: 100
Row 17: 143
Row 18: 85
Row 19: 104
Row 20: 105
Row 21: 150
Row 22: 77
Row 23: 115
Row 24: 98
Row 25: 163
Row 26: 134
Row 27: 107
Row 28: 85
Row 29: 187
Row 30: 102
Row 31: 165
Row 32: 99
Row 33: 100
Row 34: 154
Row 35: 164
Row 36: 82
Row 37: 95
Row 38: 92
Row 39: 103
Row 40: 112
Row 41: 94
Row 42: 107
Row 43: 70
Row 44: 147
Row 45: 146
Row 46: 121
Row 47: 76
Row 48: 82
Row 49: 99
Row 50: 98
Row 51: 77
Row 52: 147
Row 53:

Row 680: 143
Row 681: 56
Row 682: 108
Row 683: 103
Row 684: 124
Row 685: 96
Row 686: 168
Row 687: 176
Row 688: 112
Row 689: 77
Row 690: 121
Row 691: 105
Row 692: 117
Row 693: 85
Row 694: 108
Row 695: 79
Row 696: 127
Row 697: 85
Row 698: 91
Row 699: 141
Row 700: 102
Row 701: 172
Row 702: 105
Row 703: 79
Row 704: 83
Row 705: 125
Row 706: 96
Row 707: 50
Row 708: 99
Row 709: 123
Row 710: 144
Row 711: 109
Row 712: 165
Row 713: 169
Row 714: 79
Row 715: 124
Row 716: 131
Row 717: 91
Row 718: 164
Row 719: 103
Row 720: 96
Row 721: 72
Row 722: 112
Row 723: 89
Row 724: 84
Row 725: 143
Row 726: 121
Row 727: 103
Row 728: 92
Row 729: 105
Row 730: 141
Row 731: 100
Row 732: 95
Row 733: 100
Row 734: 107
Row 735: 106
Row 736: 97
Row 737: 119
Row 738: 106
Row 739: 114
Row 740: 105
Row 741: 61
Row 742: 105
Row 743: 97
Row 744: 99
Row 745: 76
Row 746: 79
Row 747: 139
Row 748: 153
Row 749: 119
Row 750: 60
Row 751: 107
Row 752: 124
Row 753: 78
Row 754: 82
Row 755: 90
Row 756: 116
Row 757: 144
Row 758: 104
Row

Row 1356: 102
Row 1357: 136
Row 1358: 93
Row 1359: 66
Row 1360: 100
Row 1361: 100
Row 1362: 86
Row 1363: 124
Row 1364: 175
Row 1365: 80
Row 1366: 132
Row 1367: 94
Row 1368: 94
Row 1369: 102
Row 1370: 108
Row 1371: 144
Row 1372: 121
Row 1373: 93
Row 1374: 88
Row 1375: 38
Row 1376: 138
Row 1377: 106
Row 1378: 169
Row 1379: 117
Row 1380: 97
Row 1381: 96
Row 1382: 112
Row 1383: 120
Row 1384: 78
Row 1385: 98
Row 1386: 174
Row 1387: 180
Row 1388: 72
Row 1389: 89
Row 1390: 89
Row 1391: 130
Row 1392: 80
Row 1393: 117
Row 1394: 95
Row 1395: 83
Row 1396: 116
Row 1397: 99
Row 1398: 95
Row 1399: 99
Row 1400: 165
Row 1401: 78
Row 1402: 127
Row 1403: 136
Row 1404: 100
Row 1405: 140
Row 1406: 108
Row 1407: 99
Row 1408: 153
Row 1409: 133
Row 1410: 38
Row 1411: 155
Row 1412: 113
Row 1413: 174
Row 1414: 178
Row 1415: 123
Row 1416: 77
Row 1417: 84
Row 1418: 95
Row 1419: 139
Row 1420: 158
Row 1421: 137
Row 1422: 72
Row 1423: 93
Row 1424: 103
Row 1425: 93
Row 1426: 118
Row 1427: 101
Row 1428: 122
Row 1429:

Row 1978: 148
Row 1979: 63
Row 1980: 104
Row 1981: 94
Row 1982: 91
Row 1983: 153
Row 1984: 110
Row 1985: 82
Row 1986: 93
Row 1987: 122
Row 1988: 132
Row 1989: 105
Row 1990: 124
Row 1991: 121
Row 1992: 123
Row 1993: 82
Row 1994: 111
Row 1995: 76
Row 1996: 121
Row 1997: 98
Row 1998: 127
Row 1999: 86
Row 2000: 103
Row 2001: 125
Row 2002: 98
Row 2003: 94
Row 2004: 107
Row 2005: 59
Row 2006: 158
Row 2007: 122
Row 2008: 83
Row 2009: 118
Row 2010: 112
Row 2011: 117
Row 2012: 81
Row 2013: 127
Row 2014: 116
Row 2015: 103
Row 2016: 66
Row 2017: 54
Row 2018: 98
Row 2019: 90
Row 2020: 88
Row 2021: 128
Row 2022: 121
Row 2023: 145
Row 2024: 95
Row 2025: 178
Row 2026: 95
Row 2027: 124
Row 2028: 73
Row 2029: 82
Row 2030: 123
Row 2031: 112
Row 2032: 103
Row 2033: 106
Row 2034: 120
Row 2035: 178
Row 2036: 141
Row 2037: 96
Row 2038: 153
Row 2039: 124
Row 2040: 111
Row 2041: 122
Row 2042: 99
Row 2043: 143
Row 2044: 104
Row 2045: 79
Row 2046: 148
Row 2047: 102
Row 2048: 104
Row 2049: 100
Row 2050: 81
Row 2

...

Row 85930: 109
Row 85931: 115
Row 85932: 129
Row 85933: 118
Row 85934: 121
Row 85935: 121
Row 85936: 126
Row 85937: 119
Row 85938: 124
Row 85939: 120
Row 85940: 125
Row 85941: 127
Row 85942: 125
Row 85943: 31
Row 85944: 119
Row 85945: 125
Row 85946: 126
Row 85947: 131
Row 85948: 123
Row 85949: 124
Row 85950: 117
Row 85951: 115
Row 85952: 124
Row 85953: 117
Row 85954: 110
Row 85955: 118
Row 85956: 125
Row 85957: 123
Row 85958: 120
Row 85959: 123
Row 85960: 122
Row 85961: 121
Row 85962: 122
Row 85963: 125
Row 85964: 118
Row 85965: 120
Row 85966: 124
Row 85967: 123
Row 85968: 115
Row 85969: 51
Row 85970: 117
Row 85971: 113
Row 85972: 120
Row 85973: 123
Row 85974: 123
Row 85975: 122
Row 85976: 115
Row 85977: 111
Row 85978: 121
Row 85979: 127
Row 85980: 121
Row 85981: 125
Row 85982: 120
Row 85983: 113
Row 85984: 113
Row 85985: 122
Row 85986: 129
Row 85987: 123
Row 85988: 113
Row 85989: 119
Row 85990: 116
Row 85991: 123
Row 85992: 128
Row 85993: 69
Row 85994: 122
Row 85995: 116
Row 85996: 12

Row 86511: 119
Row 86512: 117
Row 86513: 116
Row 86514: 108
Row 86515: 118
Row 86516: 122
Row 86517: 123
Row 86518: 126
Row 86519: 116
Row 86520: 119
Row 86521: 126
Row 86522: 116
Row 86523: 116
Row 86524: 125
Row 86525: 124
Row 86526: 115
Row 86527: 116
Row 86528: 119
Row 86529: 114
Row 86530: 119
Row 86531: 119
Row 86532: 119
Row 86533: 118
Row 86534: 115
Row 86535: 121
Row 86536: 122
Row 86537: 115
Row 86538: 126
Row 86539: 124
Row 86540: 119
Row 86541: 114
Row 86542: 121
Row 86543: 115
Row 86544: 126
Row 86545: 121
Row 86546: 122
Row 86547: 131
Row 86548: 126
Row 86549: 124
Row 86550: 124
Row 86551: 123
Row 86552: 120
Row 86553: 122
Row 86554: 117
Row 86555: 107
Row 86556: 116
Row 86557: 119
Row 86558: 125
Row 86559: 121
Row 86560: 97
Row 86561: 112
Row 86562: 118
Row 86563: 131
Row 86564: 123
Row 86565: 120
Row 86566: 116
Row 86567: 68
Row 86568: 115
Row 86569: 118
Row 86570: 120
Row 86571: 120
Row 86572: 126
Row 86573: 118
Row 86574: 116
Row 86575: 121
Row 86576: 119
Row 86577: 1

Row 87103: 119
Row 87104: 104
Row 87105: 125
Row 87106: 121
Row 87107: 128
Row 87108: 125
Row 87109: 126
Row 87110: 120
Row 87111: 125
Row 87112: 123
Row 87113: 115
Row 87114: 119
Row 87115: 122
Row 87116: 115
Row 87117: 120
Row 87118: 120
Row 87119: 119
Row 87120: 121
Row 87121: 117
Row 87122: 114
Row 87123: 116
Row 87124: 117
Row 87125: 121
Row 87126: 120
Row 87127: 122
Row 87128: 120
Row 87129: 114
Row 87130: 116
Row 87131: 121
Row 87132: 125
Row 87133: 118
Row 87134: 122
Row 87135: 119
Row 87136: 117
Row 87137: 121
Row 87138: 115
Row 87139: 117
Row 87140: 112
Row 87141: 121
Row 87142: 125
Row 87143: 121
Row 87144: 116
Row 87145: 119
Row 87146: 119
Row 87147: 124
Row 87148: 107
Row 87149: 122
Row 87150: 118
Row 87151: 116
Row 87152: 119
Row 87153: 122
Row 87154: 117
Row 87155: 127
Row 87156: 116
Row 87157: 116
Row 87158: 99
Row 87159: 117
Row 87160: 117
Row 87161: 119
Row 87162: 118
Row 87163: 124
Row 87164: 122
Row 87165: 124
Row 87166: 129
Row 87167: 123
Row 87168: 125
Row 87169: 

Average heartbeat profile per heartbeat category for full 187-sample set:
Global database:[0.89035972 0.75816035 0.42397223 0.21910394 0.20112697 0.21039927
 0.20580816 0.20177281 0.19869117 0.1967566  0.19877841 0.20354978
 0.2087762  0.21288461 0.21839329 0.22496562 0.23137698 0.23712264
 0.24258153 0.24792292 0.25374944 0.25974764 0.26624423 0.27273402
 0.27935471 0.28558835 0.29180828 0.29767213 0.30338428 0.30879455
 0.31389042 0.31845409 0.32208664 0.32489063 0.3267371  0.32781718
 0.32793125 0.32674565 0.32432614 0.32053665 0.31623372 0.31140718
 0.30652003 0.30169643 0.29708656 0.29222547 0.2878375  0.28396479
 0.28072441 0.27805083 0.27643191 0.27513152 0.2741522  0.27300394
 0.27235413 0.27169629 0.27141262 0.27127579 0.27159021 0.27230681
 0.27343617 0.27449172 0.27557001 0.27682303 0.27899332 0.28134854
 0.28213929 0.28022511 0.27654692 0.27206449 0.26800648 0.26423853
 0.26090975 0.2570816  0.25471175 0.25415501 0.25564617 0.2578127
 0.25983044 0.25995755 0.25852805 0.2565

C:\Users\Arnaldo\AppData\Local\Temp\ipykernel_2472\1435497785.py:276: RuntimeWarning: invalid value encountered in true_divide
  print(f'Category 4:{accumBeatCat4 / contValidSamplesCat4}')
